In [10]:
# 將 training data 的 pickle 檔載入
import pickle
import pandas as pd
import monpa as mp
import numpy as np
import jieba
import os
import tqdm




In [11]:
def batch_cut(arr,n):
    arr=np.array(arr)
    l=len(arr)
    if n>l:
        return [arr]
    result=[]
    rg=[0,n]
    while l>0:
        result.append(list(arr[rg[0]:min(rg[1],len(arr))]))
        l-=n
        rg[0]+=n
        rg[1]+=n
    
    return result


def data_gen(cts):
    tg=['Dfa','Dfb','Nv','VA','VAC','VB','VC','VCL','VD','VE','VF','VG','VH','VHC','VI','VJ','VK','VL','Na','A','Nc','Ncd','ORG','PER']
    result=[]
    psegs_ct=[]
    bach_size=32
    baches=batch_cut(cts,bach_size)
    for bach in tqdm(baches):
        #print(type(bach))
        pseg_cts=mp.pseg_batch(bach)
        for pseg_ct in pseg_cts:
            chosen_words=[]
            
            for pseg in pseg_ct:
                try:
                    if pseg[1] in tg:
                        chosen_words.append(pseg[0].strip())
                except:
                    print(pseg)
            #print(chosen_words)
            result.append(" ".join(chosen_words))
    return(result)


a=[2,3,4,5,6]
batch_cut(a,2)

[[2, 3], [4, 5], [6]]

In [4]:
import json

def check_dict(dictionary,w):
    try:
        return dictionary[w]
    except:
        v=len(dictionary)+1
        dictionary[w]=v
        return v
    



In [5]:

def data_gen(dataset,file_name):
    #load dictionary
    dictionary = {}
    try:
        with open('dictionary.json', 'r') as fp:
            dictionary = json.load(fp)
    except:
        pass
    #build data
    dataset=np.array(dataset)
    did_dataset={'Label':[],'did_content':[]}
    for data in dataset:
        did_dataset['Label'].append(data[0])
        ct=[]
        for word in data[1].split():
            ct.append(str(check_dict(dictionary,word)))
        did_dataset['did_content'].append(' '.join(ct))
    pd.DataFrame(did_dataset).to_csv(file_name)
    #Save dictionary
    with open('dictionary.json', 'w') as fp:
            json.dump(dictionary, fp)
    return pd.DataFrame(did_dataset)

train_df = pickle.load(open("./monpa_train.pickle", "rb"))



data_gen(train_df[:3000],'test_dataset100.csv')
data_gen(train_df[3000:],'train_dataset100.csv')



,Label,did_content
0,tech,32674 10942 14092 2789 2956 335 15430 339 713 ...
1,health,41446 41447 29073 1368 24594 108 1366 1431 143...
2,edu,41455 4580 41456 1939 41456 10905 4580 1939 76...
3,tech,41465 3205 2477 391 1344 41466 1927 828 2199 2...
4,travel,5054 360 41475 19897 5724 5004 42 7095 366 367...
...,...,...
26995,health,155 531 21765 3157 2200 1738 49378 1717 1431 1...
26996,health,36610 154 696 7117 839 967 1841 279 312 4077 8...
26997,health,2304 376 149375 9141 328 8335 616 154 97 839 3...
26998,travel,793 360 149376 41329 1400 229 4625 33666 1102 ...


In [6]:
test_df = pickle.load(open("./monpa_test.pickle", "rb"))
test_df


,ID,Content
0,1,基督城 待 重建 外交部 籲 勿 前往 （ 中央社 記者 曾依璇 台北 24...
1,2,成龍 溼地 裝置 藝術 貼近 土地 生態 ( 影音 ) 台灣 西南 沿海 地...
2,3,尼克 教頭 確認 豪哥 本 季 報銷 林書豪 到底 會不會 提早 復出 為 尼克隊 上...
3,4,台灣 智庫 新書 質疑 馬強 推 ECFA 有 不能 說 的 秘密 〔 記者 ...
4,5,宋批 台灣 共識 挑釁 會 引共憤 （ 中央社 記者 卞金峰 桃園縣 1 日 電 ） ...
...,...,...
19995,19996,失 球后 曾雅妮 ： 要 開始 追逐 世界 第一 了 新 頭殼 newtalk 2...
19996,19997,看 經典賽 南韓 稱 我 最 具 威脅 南韓 電視 在 球評 分析 說 ： 日本 「...
19997,19998,英文 旺報 今 創刊 走向 全 世界 旺報 【 記者 盧伯華 ／ 台北 報導 ...
19998,19999,山 櫻花 盛開 奧萬 大紅 通通 〔 自由時報 記者 佟振國 ／ 仁愛 報導...


In [12]:
dictionary = {}
try:
    with open('dictionary.json', 'r') as fp:
        dictionary = json.load(fp)
except:
    pass
#build data
dataset=np.array(test_df['Content'])
did_dataset=[]
for data in dataset:
    ct=[]
    for word in data.split():
        ct.append(str(check_dict(dictionary,word)))
    did_dataset.append(' '.join(ct))
#print(did_dataset[0])
test_df['did_content']=did_dataset
#pd.DataFrame(dataset).to_csv('unknow.csv')
#Save dictionary
with open('dictionary.json', 'w') as fp:
        json.dump(dictionary, fp)
pd.DataFrame(test_df).to_csv('unknow.csv')
test_df

,ID,Content,did_content
0,1,基督城 待 重建 外交部 籲 勿 前往 （ 中央社 記者 曾依璇 台北 24...,19790 4570 4820 1188 4045 16959 1191 35 661 36...
1,2,成龍 溼地 裝置 藝術 貼近 土地 生態 ( 影音 ) 台灣 西南 沿海 地...,19804 15010 7000 2142 19805 1748 1158 932 3047...
2,3,尼克 教頭 確認 豪哥 本 季 報銷 林書豪 到底 會不會 提早 復出 為 尼克隊 上...,3206 2187 5520 19820 1491 2399 10692 464 12596...
3,4,台灣 智庫 新書 質疑 馬強 推 ECFA 有 不能 說 的 秘密 〔 記者 ...,519 10594 19299 886 19830 712 11163 109 2064 7...
4,5,宋批 台灣 共識 挑釁 會 引共憤 （ 中央社 記者 卞金峰 桃園縣 1 日 電 ） ...,19839 519 10080 8450 631 19840 35 661 368 1984...
...,...,...,...
19995,19996,失 球后 曾雅妮 ： 要 開始 追逐 世界 第一 了 新 頭殼 newtalk 2...,4030 3875 3867 213 154 912 3079 1107 867 844 7...
19996,19997,看 經典賽 南韓 稱 我 最 具 威脅 南韓 電視 在 球評 分析 說 ： 日本 「...,586 20702 3885 3748 1452 121 581 6878 3885 312...
19997,19998,英文 旺報 今 創刊 走向 全 世界 旺報 【 記者 盧伯華 ／ 台北 報導 ...,1924 489 1575 56066 15770 1294 1107 489 366 36...
19998,19999,山 櫻花 盛開 奧萬 大紅 通通 〔 自由時報 記者 佟振國 ／ 仁愛 報導...,3758 3609 363 77709 9972 4092 660 2003 368 166...


In [8]:
dataset=np.array(test_df['Content'])
dataset


array(['基督城 待 重建     外交部 籲 勿 前往     （ 中央社 記者 曾依璇 台北 24 日 電 ） 外交部 今天 表示 ， 紐西蘭 南島 基督城 在 強震 後 ， 市中心 80 % 已 無 供水 ， 且 交通 仍 管制 中 ， 台灣 民眾 短期 內 不宜 前往 商旅 。 芮氏 規模 6.3 的 強震 於 22日 襲擊 基督城 ， 造成 嚴重 傷亡 ， 郵政 系統 暫停 服務 ， 電力 雖 恢復 了 60 % ， 道路 交通 仍 管制 中 。 由於 災 後 重建 ， 百廢 待舉 ， 外交部 建議 ， 民眾 短期 內 不宜 前往 基督城 商旅 。 外交部 表示 ， 在 紐西蘭 境 內 的 民眾 ， 應 保持 警覺 ， 隨時',
       '成龍 溼地 裝置 藝術   貼近 土地 生態 ( 影音 )       台灣 西南 沿海 地區 ， 因 養殖業 長期 抽取 地 下水 ， 導致 嚴重 地層 下陷 。 其中 雲林縣口湖鄉 成龍村 ， 因 民國 七十五年 的 韋恩 颱風 ， 造成 當地 海水 倒灌 ， 積水 不 退 至今 成為 一 塊 溼地 ， 最近 要 透過 來自 國外 的 藝術家 ， 利用 當地 的 素材 和 地方 志工 、 學生 ， 共同 完成 五 件 裝置 藝術 ， 除了 要 推廣 環境 保護 外 ， 也 要 設法 改善 地層 下陷 的 問題 。 這 ㄧ 大 片 溼地 ， 位在 雲林縣口湖鄉 成龍村 ， 面積 大約 有 100 公頃 ， 不過 它 形成',
       '尼克 教頭 確認  豪哥 本 季 報銷   林書豪 到底 會不會 提早 復出 為 尼克隊 上 場 打 對 熱火 的 第五 場 季後賽 ， 吊足 觀眾 的 胃口 ， 尼克隊 總教頭 伍森 接受 東森 駐 紐約 記者 的 訪問 時 ， 非常 確定 地 說 ， 林書豪 本 季 是 來不及 復出 。 不過 ESPN 專欄 作家 則 是 指出 ， 尼克 高層 很 心急 ， 第五 戰 豪小子 出戰 機會 高達 9成5 ！ 結果 如何 ？ 要 等到 比賽 才 會 揭曉 。 尼克隊 將 在 台北 時間 5月 10 號 早上 在 客場 對上 熱火隊 ， 由於 後衛 傷兵 太多 ， 林書豪 會 不會 扮演 尼克隊 的 救世主 角色 ， 下',
       ...,
       '英文 

In [13]:
len(dictionary)

207930